# Generating Data

In PyAmpTools, `gen_amp` is a class that uses `AmpTools` to generate events based on some set of amplitudes. `gen_amp` is originally a compiled program, [source](https://github.com/JeffersonLab/halld_sim/blob/master/src/programs/Simulation/gen_amp/gen_amp.cc). AmpTools uses a configuration file to specify the model and dataset to be fitted.


In order to use `gen_amp` within PyAmpTools you must first build the library. Simply go to `external/AMPTOOLS_GENERATORS` and run `make`

In [ ]:
import ROOT
import os
from pyamptools import atiSetup

REPO_HOME = os.environ["REPO_HOME"]
USE_MPI, USE_GPU, RANK_MPI = atiSetup.setup(globals(), use_genamp=True)  # RANK_MPI defaults to 0 even if not using MPI
gen_amp = ROOT.gen_amp
TFile = ROOT.TFile

## Configuration File

A configuration file contains the specifications of the model (and data) and contains all the information needed by `AmpTools` to generate Monte Carlo data and to perform fits. 

In AmpTools, an amplitude belongs in a coherent sum which itself belongs in to reaction. A unique name for an amplitude follows the format ~ `reactionName::sumName::ampName` where double colons are used to separate the terms. 

If the construction of the amplitudes is complicated with many repeating formats, then a configuration file generator could be used. See [cfgGenerator](https://lan13005.github.io/PyAmpTools/api/cfgGenerator.html) for API and source code.

For now, we can use this generated cfg file

In [ ]:
cfgfile = f"{REPO_HOME}/tests/samples/SIMPLE_EXAMPLE/fit.cfg"

In [ ]:
%cat $cfgfile

Some things to note. The head of the cfg file contains available keywords that provide some specific functionality. See [AmpTools Documentation](https://github.com/mashephe/AmpTools/blob/master/AmpTools_User_Guide.pdf) for more information. The amplitudes used here are known as the Zlm amplitudes. See here for the [Zlm amplitude implementation](https://halldweb.jlab.org/DocDB/0040/004094/003/amptools_polarization.pdf). These complex parameters are initialized in cartesian coordinates. Polar coordinates are are possible

## Generating Events

With the configuration file we can request `gen_amp` to make some simulations.  Not all lines in the above cfg file is needed for generation, for instance the dataset locations, but will be used during fitting.

Here, `gen_amp` is class, and requires instantiation

In [ ]:
generator = gen_amp()

`gen_amp` contains many variables to condition the generator. We can see these attribute below

In [ ]:
generator.print_help()

A good way to set these data members is to create a dictionary to store our specifications (and save if we wish for reproducibility/referencing) and condition our generator as follows. 

These data members requires modification
1. configfile
2. outname

In [ ]:
outfile = "test.root"

conditions = {
    "configfile": cfgfile,
    "outname": outfile,
    "nEvents": 1000,
}

for k, v in conditions.items():
    setattr(generator, k, v)

To see the values of all the datamembers (most contain some default values):

In [ ]:
for member in generator.data_members:
    member = str(member)
    print(f"{member:<20} {getattr(generator, member )}")

After conditioning the generator, simulations can be performed!

In [ ]:
generator.generate()

A `*diagnostic.root` is also created containing a variety of mass and angular histograms. A file called `test.root` should have been created containing the 4-momentum components for all particles. To understand physical data collected by a detector, simulations like this one would have to then be passed through a simulation of detector to produce realistic data. We can use `PyROOT` to quickly glance at the contents of the file.

In [ ]:
file = ROOT.TFile(outfile)
tree = file.kin

In [ ]:
tree.Print()

Lets clean up these generated files as we do not actually need them

In [ ]:
# Free the file handle first so we can clean up
del tree
del file

!rm gen_amp_diagnostic.root
!rm test.root
!rm local_beam.conf